In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hypermri.utils.utils_fitting as utf
import hypermri.utils.utils_spectroscopy as uts
import hypermri.utils.utils_general as utg
from tqdm.auto import tqdm
import ipywidgets as widgets
import matplotlib
import os
import sys
matplotlib.rcParams.update({'font.size': 12,'font.family':'serif','font.serif':['Computer Modern'],"text.usetex" : True,})
# Get the current working directory
cwd = os.getcwd()
parent_dir = os.path.dirname(os.path.dirname(cwd))
sys.path.append(parent_dir)
# define paths:
sys.path.append('../')
import TEMPLATE
# get paths:
repopath, basepath, savepath,publication_path = TEMPLATE.import_all_packages(True)
%matplotlib widget

# Functions

In [ ]:
def apply_lb(fid,time_ax,lb):
    return fid*np.exp(-np.pi*lb*time_ax)

def compute_snr(raw_spec,fit_spectrums,ppm_range,plot=False):
    from scipy.stats import rayleigh
    peak_snrs=np.ones((4,noise_vals,repeats))*np.nan
    noise_vals=noise_levels.shape[0]
    
    for repeat in range(repeats):
        
        if plot:
            plt.close('all')

            fig,ax=plt.subplots(noise_vals,2,tight_layout=True,figsize=(6,10))
            noise_floor=np.ones((noise_vals))
            for n in range(noise_vals):
                ax[n,0].plot(np.abs((np.squeeze(raw_spec)[:,n,repeat]-np.mean(np.abs(np.squeeze(raw_spec)[0:150,n,repeat])))/np.std(np.abs(np.squeeze(raw_spec)[0:150,n,repeat]))))
                ax[n,0].plot(np.abs((np.squeeze(raw_spec)[:,n,repeat]-np.mean(np.abs(np.squeeze(raw_spec)[0:150,n,repeat])))/np.std(np.abs(np.squeeze(raw_spec)[0:150,n,repeat])))[0:150])
                noise_spec=np.abs(((np.squeeze(raw_spec)[:,n,repeat]-np.mean(np.real(np.squeeze(raw_spec)[0:150,n,repeat])))/np.std(np.real(np.squeeze(raw_spec)[0:150,n,repeat]))))[0:150]

                params = rayleigh.fit(noise_spec)
                scale = params[1]

                # Define x values for plotting the fitted distribution
                x = np.linspace(0, max(noise_spec), 100)
                pdf_fitted = rayleigh.pdf(x, loc=0, scale=scale)


                ax[n,1].hist(noise_spec, bins=30, density=True, alpha=0.3, color='C0', edgecolor='black')
                ax[n,1].plot(x, pdf_fitted, 'r-', label=f'Rayleigh fit (scale={scale:.2f})')
                # Create a frozen Rayleigh distribution object with the fitted scale parameter

                fitted_rayleigh = rayleigh(scale=scale)

                # Get mean and standard deviation
                mean = fitted_rayleigh.mean()
                std_dev = fitted_rayleigh.std()
                noise_floor[n]=mean+std_dev
                ax[n,1].set_title(str(mean.round(1))+'±'+str(std_dev.round(1)))

                fig,ax=plt.subplots(noise_vals,2,tight_layout=True,figsize=(7,5*noise_vals/2))

                mean_noise=[np.mean(np.abs(np.squeeze(raw_spec)[0:150,n,repeat])) for n in range(noise_vals)]
                std_noise=[np.std(np.abs(np.squeeze(raw_spec)[0:150,n,repeat])) for n in range(noise_vals)]

                for noise_val in range(noise_vals):
                    ax[noise_val,0].plot(ppm_range,np.abs(np.squeeze(raw_spec))[:,noise_val,repeat])
                    ax[noise_val,0].plot(ppm_range,np.sum(np.abs(np.squeeze(fit_spectrums)[:,noise_val,repeat,:]),axis=1),color='r')
                    ax[noise_val,1].plot(ppm_range,np.abs(np.squeeze(raw_spec))[:,noise_val,repeat],color='k',alpha=0.3)

                    for peak in range(4):
                        max_peak_fit_val=np.max(np.abs(np.squeeze(fit_spectrums)[:,noise_val,repeat,peak]))
                        snr=np.round((max_peak_fit_val-mean_noise[noise_val])/std_noise[noise_val],2)
                        peak_snrs[peak,noise_val,repeat]=snr
                        ax[noise_val,1].plot(ppm_range,np.abs(np.squeeze(fit_spectrums)[:,noise_val,repeat,peak]),label=metabs[peak]+',SNR='+str(snr))
                        ax[noise_val,1].set_title('Noise='+str(np.round(noise_floor[noise_val],0)))
                    ax[noise_val,0].set_xlim([195,155])
                    ax[noise_val,1].set_xlim([195,155])

                    ax[noise_val,1].legend()
                    ax[noise_val,0].set_title('Noise val='+str(noise_levels[noise_val]))

        else:
            noise_floor=np.ones((noise_vals))
            for n in range(noise_vals):
                noise_spec=np.abs(((np.squeeze(raw_spec)[:,n,repeat]-np.mean(np.real(np.squeeze(raw_spec)[0:150,n,repeat])))/np.std(np.real(np.squeeze(raw_spec)[0:150,n,repeat]))))[0:150]
                mean_noise=[np.mean(np.abs(np.squeeze(raw_spec)[0:150,n,repeat])) for n in range(noise_vals)]
                std_noise=[np.std(np.abs(np.squeeze(raw_spec)[0:150,n,repeat])) for n in range(noise_vals)]
                for noise_val in range(noise_vals):
                    for peak in range(4):
                        max_peak_fit_val=np.max(np.abs(np.squeeze(fit_spectrums)[:,noise_val,repeat,peak]))
                        snr=np.round((max_peak_fit_val-mean_noise[noise_val])/std_noise[noise_val],2)
                        peak_snrs[peak,noise_val,repeat]=snr
        
    return peak_snrs,noise_floor


# General parameters

In [ ]:
bandwidth = 5000
dwell_time = 1/bandwidth
b0=3
fieldoffset=170.4

# 1. SNR Simulation
### 2048 points, 50 SNR Levels, 100 repetitions

In [ ]:
samplings = [2048]
repeats = 100
noise_base_level=0.01
noise_levels=np.linspace(noise_base_level,1,50)
scatterpointsize=10

## Read in fitted data

In [ ]:
data = utg.load_as_pkl('','sim_SNR_more_vals_rev1.pkl')
peak_snrs=data['snrs']
temps_truth=data['temps_truth']
temps_fit=data['temps_fit']
noise_levels=data['noise_levels']
repeats=data['repeats']

In [ ]:
revisionpath=publication_path+'Revision1/SI_Figures/'

In [ ]:
fig,ax=plt.subplots(1,figsize=(6.9/2,3),tight_layout=True)

ax.set_ylabel('$ \sigma (\Delta T) [^\circ C]$')
ax.scatter(np.mean(peak_snrs,axis=1),np.std(temps_truth.squeeze()-temps_fit.squeeze(),axis=1),color='k',s=scatterpointsize)
ax.set_xlabel('SNR (pyruvate)')
ax.set_xlim([0,100])
ax.hlines(0,0,100,color='w',linestyle='solid')


# 2. Linebroadening

In [ ]:
linebroadenings=np.linspace(0,100,100)
samplings = [2048]
noise_base_level=0.1
repeats=100

## Load simulated data

In [ ]:
data = utg.load_as_pkl('','sim_varying_lb.pkl')
peak_snrs=data['snrs']
temps_truth=data['temps_truth']
temps_fit=data['temps_fit']

linebroadenings=data['linebroadenings']

In [ ]:
fig,ax=plt.subplots(1,figsize=(6.9/2,3),tight_layout=True)
ax.scatter(linebroadenings,np.std(temps_truth.squeeze()-temps_fit.squeeze(),axis=1),color='k',s=scatterpointsize)
ax.set_xlabel('Lb [Hz]')
ax.set_ylabel('$\sigma (\Delta T) [^\circ C]$')


In [ ]:
fig,ax=plt.subplots(1,figsize=(6.9/2,3),tight_layout=True)
ax.scatter(linebroadenings,np.mean(temps_fit.squeeze(),axis=1)-np.mean(temps_truth.squeeze(),axis=1),color='k',s=scatterpointsize)
ax.set_xlabel('Lb [Hz]')
ax.set_ylabel('$ T_{fit}-T_{true} [^\circ C]$')


In [ ]:
fig,ax=plt.subplots(1,figsize=(6.9/2,3),tight_layout=True)
ax.scatter(linebroadenings,np.std(temps_truth.squeeze()-temps_fit.squeeze(),axis=1),color='k',s=scatterpointsize,label='$\sigma (T_{fit}-T_{true})$')
ax.set_xlabel('Lb [Hz]')
ax.set_ylabel('$\sigma (\Delta T) [^\circ C]$',color='k')
ax.tick_params(axis='y', labelcolor='k')

ax2=ax.twinx()
ax2.scatter(linebroadenings,np.mean(temps_fit.squeeze(),axis=1)-np.mean(temps_truth.squeeze(),axis=1),color='r',s=scatterpointsize,label='$T_{fit}-T_{true}$')
ax2.set_xlabel('Lb [Hz]')
ax2.set_ylabel('$ \Delta T [^\circ C]$',color='r')
ax2.tick_params(axis='y', labelcolor='r')


# 3. T2* Simulations

In [ ]:
bandwidth = 5000
dwell_time = 1/bandwidth
samplings = [2048]
repeats = 100
noise_base_level=0.01
b0=3
fieldoffset=170.4
#t2s=np.linspace(0.002,0.03,50)
fwhms=np.arange(5,150,5)
t2s=1/(np.pi*fwhms)


In [ ]:
data = utg.load_as_pkl('','sim_varying_t2.pkl')
peak_snrs=data['snrs']
temps_truth=data['temps_truth']
temps_fit=data['temps_fit']
repeats=data['repeats']
t2vals=data['t2s']

In [ ]:
fig,ax=plt.subplots(1,figsize=(6.9/2,3),tight_layout=True)


std_err=np.std(data['temps_fit'].squeeze()-data['temps_truth'].squeeze(),axis=1)
ax.scatter(1/(np.pi*t2vals),std_err,color='k',s=10)
ax.set_xlabel('FWHM [Hz]')

ax.set_ylabel('$\sigma (\Delta T) [^\circ C]$')



# 4. Sampling simulations

In [ ]:
data = utg.load_as_pkl('','sim_sampling_rev1.pkl')
peak_snrs=data['snrs']
temps_truth=data['temps_truth']
temps_fit=data['temps_fit']
repeats=data['repeats']
samplings=data['samplings']


In [ ]:
fig,ax=plt.subplots(1,figsize=(6.9/2,3),tight_layout=True)
for index,sampling in enumerate(samplings):
    ax.set_ylabel('$ \sigma (\Delta T) [^\circ C]$')
    ax.scatter(1/(bandwidth/sampling),np.std(temps_truth.squeeze()[:,index]-temps_fit.squeeze()[:,index],axis=0),c='k',s=scatterpointsize)
    ax.set_xlabel('Spectral resolution [pts/Hz]')


In [ ]:
fig,ax=plt.subplots(1,figsize=(6.9/2,3),tight_layout=True)
import matplotlib.ticker as ticker

for index,sampling in enumerate(samplings):
    ax.set_ylabel('$ \sigma (\Delta T) [^\circ C]$')
    ax.scatter((bandwidth/sampling),np.std(temps_truth.squeeze()[:,index]-temps_fit.squeeze()[:,index],axis=0),c='k',s=10)
    ax.set_xlabel('Spectral resolution [Hz/pt]')
    ax.set_xscale('log')
    ax.set_yscale('log')
    formatter = ticker.ScalarFormatter()
    formatter.set_scientific(False)
    ax.yaxis.set_major_formatter(formatter)
    ax.xaxis.set_major_formatter(formatter)
 